In [1]:
%%capture
## loading env variables
%env HF_HOME=/hpcwork/ba214121/.cache/huggingface
%env TRANFORMERS_CACHE=/hpcwork/ba214121/.cache/huggingface
%env HF_DATASETS_CACHE=/hpcwork/ba214121/.cache/huggingface
!pip install -U datasets

!echo $HF_DATASETS_CACHE

In [2]:
## getting dataset from HF

import os
from datasets import load_dataset

base_sep_dataset=load_dataset("ruggsea/stanford-encyclopedia-of-philosophy_instruct")

In [3]:
# using chat openai to get completions
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def generate(prompt:str):
    response=client.chat.completions.create(
    model="mistralai/Mistral-Large-Instruct-2407",
    messages=[
        {"role": "user", "content":prompt}
    ],
    temperature=1,
    max_tokens=8192
    ).choices[0].message.content
    return response

#generate("what are you")

In [4]:
with open("multi_turn_gen_prompt.txt","r") as f:
    prompt_template=f"{f.read()}"

def get_random_prompt(n_turns, level, sep):
    template=prompt_template.format(n_turns=n_turns, level=level, sep=sep)
    return template

print(get_random_prompt(1, "grad", "ciao"))

You are an expert and well-read Philosophy professor at a prestigious university named Phil. You are known for your deep understanding of the subject and your ability to explain complex concepts in advanced yet accessible ways, sometimes using specific jargon but always making sure your students understand the material. Your teaching style is characterized by the following traits:

Analogy aficionado: You frequently use creative analogies to explain abstract concepts.
Devil's advocate: You enjoy presenting counterarguments to challenge students' thinking.
Practical philosophy proponent: You emphasize the real-world applications of philosophical concepts.
Socratic method enthusiast: Even if often you don't directly ask questions to your students, you sometimes guide them using hints to help them understand specific concepts better.

You are recalling a conversation you had with a student who is at the grad level and usually inquires using short, simple but interesting questions/remarks.

In [5]:
# each template will need a number of back and forth turns from 1 to 10, with an average of 1.5,
# an excerpt from the SEP,
# a grad level to base the conversation on (starting undergrad, master level, phd level), with undergrad being the most common (70%)

import random

def get_random_turns():
    # get a randomly distributed number of turns, with an average of 1.5, betwwen 1 and 15. Gaussian with long tail
    while True:
        n_turns=round(random.gauss(1.5,3))
        if n_turns>0 and n_turns<8:
            return n_turns

def get_random_level():
    # get a randomly distributed level, with undergrad being the most common
    while True:
        level=random.choices(["undergrad","master","phd"],weights=[0.7,0.2,0.1],k=1)[0]
        return level

# the output will be an array of dictionaries, each dictionary will have the following keys:
# role: user or assistant
# content: the text of the message

# they will be extracted from the generated text, with the user starting the conversation (the user is the student character), the assistant will be the teacher character named "Phil" in the prompt

def extract_conversation(text):
    # the conversation is separated by new lines, student turns start with "Student:" and teacher turns start with "Phil:" or "Professor:"
    conversation=[]
    # strip new lines at the beginning and end
    text=text.strip()
    text=text.strip("\n")
    lines=text.split("\n")
    for line in lines:
        if line.startswith("Student:"):
            conversation.append({"role":"user","content":line.replace("Student:","").strip()})
        elif line.startswith("Phil:") or line.startswith("Professor:"):
            conversation.append({"role":"assistant","content":line.replace("Phil:","").replace("Professor:","").strip()})
    
    # make sure the conversation starts with the user and alternates between user and assistant, otherwise return an empty conversation
    if len(conversation)>0 and conversation[0]["role"]=="user":
        for i in range(1,len(conversation)):
            if conversation[i]["role"]==conversation[i-1]["role"]:
                print("Conversation is not alternating between user and assistant")
                return []
    return conversation

fake_conversation="""Student: What is the meaning of life?
Phil: The meaning of life is a philosophical question concerning the significance of living or existence in general.
Student: What is the meaning of life?
Phil: The meaning of life is a philosophical question concerning the significance of living or existence in general.
"""

print(extract_conversation(fake_conversation))

[{'role': 'user', 'content': 'What is the meaning of life?'}, {'role': 'assistant', 'content': 'The meaning of life is a philosophical question concerning the significance of living or existence in general.'}, {'role': 'user', 'content': 'What is the meaning of life?'}, {'role': 'assistant', 'content': 'The meaning of life is a philosophical question concerning the significance of living or existence in general.'}]


In [6]:
# generate one example to test it

random_sep=base_sep_dataset["train"][random.randint(0,len(base_sep_dataset["train"])-1)]["answer"]

prompt=get_random_prompt(get_random_turns(),get_random_level(),random_sep)

print(prompt)

print(generate(prompt))

You are an expert and well-read Philosophy professor at a prestigious university named Phil. You are known for your deep understanding of the subject and your ability to explain complex concepts in advanced yet accessible ways, sometimes using specific jargon but always making sure your students understand the material. Your teaching style is characterized by the following traits:

Analogy aficionado: You frequently use creative analogies to explain abstract concepts.
Devil's advocate: You enjoy presenting counterarguments to challenge students' thinking.
Practical philosophy proponent: You emphasize the real-world applications of philosophical concepts.
Socratic method enthusiast: Even if often you don't directly ask questions to your students, you sometimes guide them using hints to help them understand specific concepts better.

You are recalling a conversation you had with a student who is at the undergrad level and usually inquires using short, simple but interesting questions/rem

In [7]:
answer=generate(prompt)
print(extract_conversation(answer))

[{'role': 'user', 'content': "I heard Turing had something to do with computers. What's the big deal about him?"}, {'role': 'assistant', 'content': "Ah, Turing! Imagine a chef who not only invents a revolutionary kitchen gadget but also writes the cookbook that everyone uses. Turing envisioned a 'universal machine' that could mimic any process of reasoning. Pretty much the foundation of modern computing."}, {'role': 'user', 'content': "But wasn't that just an idea? Didn't someone else actually build the first computer?"}, {'role': 'assistant', 'content': "Well, consider this—if someone describes a plane perfectly, but can't build it due to lack of materials, are they any less visionary? Turing's 'universal machine' was initially deemed impractical, but he persisted. He even started working with relays for a cryptographic machine in 1937."}, {'role': 'user', 'content': 'So, he was ahead of his time?'}, {'role': 'assistant', 'content': 'Indeed! Think of a surfer waiting for the perfect w

In [8]:
import json
import os
import aiohttp
import asyncio
from tqdm.asyncio import tqdm_asyncio

def save_conversation(conversation):
    with open("multi_turn_gen_conversations.jsonl", "a") as f:
        f.write(json.dumps(conversation) + "\n")

if os.path.exists("multi_turn_gen_conversations.jsonl"):
    with open("multi_turn_gen_conversations.jsonl", "r") as f:
        conversations = [json.loads(line) for line in f]
else:
    conversations = []

async def generate_async(session, semaphore, prompt):
    async with semaphore:
        url = "http://localhost:8000/v1/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer EMPTY",
        }
        data = {
            "model": "meta-llama/Meta-Llama-3.1-70B-Instruct",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 1,
            "max_tokens": 8192
        }

        async with session.post(url, headers=headers, json=data) as response:
            if response.status == 200:
                result = await response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"Error: {response.status}"

async def process_task(i, prompt, n_turns, session, semaphore, pbar):
    while True:
        try:
            response = await generate_async(session, semaphore, prompt)
            conversation = extract_conversation(response)  # Assuming you have this function
            if len(conversation) >= 2 * n_turns:
                conversations.append({"prompt": prompt, "conversation": conversation})
                save_conversation({"prompt": prompt, "conversation": conversation})
                pbar.update(1)
                break
            else:
                print(f"Conversation has length {len(conversation)}, less than the needed {2 * n_turns}")
        except Exception as e:
            print(f"Error processing response: {e}")
            pass


async def main():
    semaphore = asyncio.Semaphore(100)
    tasks = []
    async with aiohttp.ClientSession() as session:
        pbar = tqdm_asyncio(total=len(base_sep_dataset["train"]))
        for i, sep in enumerate(base_sep_dataset["train"]["answer"]):
            if len(conversations) > i:
                pbar.update(1)
                continue
            n_turns = get_random_turns()  # Assuming you have this function
            level = get_random_level()  # Assuming you have this function
            prompt = get_random_prompt(n_turns, level, sep)  # Assuming you have this function
            tasks.append(process_task(i, prompt, n_turns, session, semaphore, pbar))
            

        await asyncio.gather(*tasks)
        pbar.close()  # Close the progress bar

# Run the async main function in Jupyter Notebook
await main()

100%|██████████| 11904/11904 [00:00<00:00, 25912.33it/s]


In [9]:
# set up the generation to save the conversations in a file called "multi_turn_gen_conversations.jsonl" and start back from the last conversation

import json
from tqdm.notebook import tqdm

def save_conversation(conversation):
    with open("multi_turn_gen_conversations.jsonl","a") as f:
        f.write(json.dumps(conversation)+"\n")

if os.path.exists("multi_turn_gen_conversations.jsonl"):
    with open("multi_turn_gen_conversations.jsonl","r") as f:
        conversations=[json.loads(line) for line in f]
else:
    conversations=[]

# generate a valid conversation for each excerpt in the dataset

for i,sep in tqdm(enumerate(base_sep_dataset["train"]["answer"]), total=len(base_sep_dataset["train"])):
    # skip indexes already generated
    if len(conversations)>i:
        continue
    n_turns=get_random_turns()
    level=get_random_level()
    prompt=get_random_prompt(n_turns, level, sep)
    while True:
        try:
            response=generate(prompt)
            conversation=extract_conversation(response)
            # check that the conversation has at least the required number of turns
            if len(conversation)>=2*n_turns:
                conversations.append({"prompt":prompt,"conversation":conversation})
                save_conversation({"prompt":prompt,"conversation":conversation})
                break
            else:
                print(f"Conversation has lenght {len(conversation)}, less than the needed {2*n_turns}")
        except:
            pass

  0%|          | 0/11904 [00:00<?, ?it/s]

In [10]:
## now the athene version

import json
import os
import aiohttp
import asyncio
from tqdm.asyncio import tqdm_asyncio

def save_conversation(conversation):
    with open("multi_turn_gen_conversations_athene.jsonl", "a") as f:
        f.write(json.dumps(conversation) + "\n")

if os.path.exists("multi_turn_gen_conversations_athene.jsonl"):
    with open("multi_turn_gen_conversations_athene.jsonl", "r") as f:
        conversations = [json.loads(line) for line in f]
else:
    conversations = []

async def generate_async(session, semaphore, prompt):
    async with semaphore:
        url = "http://localhost:8000/v1/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer EMPTY",
        }
        data = {
            "model": "Nexusflow/Athene-70B",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 1,
            "max_tokens": 7000
        }

        async with session.post(url, headers=headers, json=data) as response:
            if response.status == 200:
                result = await response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"Error: {response.status}"

async def process_task(i, prompt, n_turns, session, semaphore, pbar):
    while True:
        try:
            response = await generate_async(session, semaphore, prompt)
            conversation = extract_conversation(response) 
            if len(conversation) >= 2 * n_turns:
                conversations.append({"prompt": prompt, "conversation": conversation})
                save_conversation({"prompt": prompt, "conversation": conversation})
                pbar.update(1)
                break
            else:
                print(f"Conversation has length {len(conversation)}, less than the needed {2 * n_turns}")
        except Exception as e:
            print(f"Error processing response: {e}")
            pass


async def main():
    semaphore = asyncio.Semaphore(500)
    tasks = []
    async with aiohttp.ClientSession() as session:
        pbar = tqdm_asyncio(total=len(base_sep_dataset["train"]))
        for i, sep in enumerate(base_sep_dataset["train"]["answer"]):
            if len(conversations) > i:
                pbar.update(1)
                continue
            if len(sep)>3000:
                sep=sep[:3000]
            n_turns = get_random_turns()
            level = get_random_level()  
            prompt = get_random_prompt(n_turns, level, sep)
            tasks.append(process_task(i, prompt, n_turns, session, semaphore, pbar))
            

        await asyncio.gather(*tasks)
        pbar.close()  # Close the progress bar

# Run the async main function in Jupyter Notebook
await main()

100%|██████████| 11904/11904 [00:00<00:00, 412752.30it/s]


In [11]:
## now the mistral-large version

import json
import os
import aiohttp
import asyncio
from tqdm.asyncio import tqdm_asyncio

def save_conversation(conversation):
    with open("multi_turn_gen_conversations_mistral_large.jsonl", "a") as f:
        f.write(json.dumps(conversation) + "\n")

if os.path.exists("multi_turn_gen_conversations_mistral_large.jsonl"):
    with open("multi_turn_gen_conversations_mistral_large.jsonl", "r") as f:
        conversations = [json.loads(line) for line in f]
else:
    conversations = []

async def generate_async(session, semaphore, prompt):
    async with semaphore:
        url = "http://localhost:8000/v1/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer EMPTY",
        }
        data = {
            "model": "mistralai/Mistral-Large-Instruct-2407",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 1,
            "max_tokens": 7000
        }

        async with session.post(url, headers=headers, json=data) as response:
            if response.status == 200:
                result = await response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"Error: {response.status}"

async def process_task(i, prompt, n_turns, session, semaphore, pbar):
    while True:
        try:
            response = await generate_async(session, semaphore, prompt)
            conversation = extract_conversation(response) 
            if len(conversation) >= 2 * n_turns:
                conversations.append({"prompt": prompt, "conversation": conversation})
                save_conversation({"prompt": prompt, "conversation": conversation})
                pbar.update(1)
                break
            else:
                print(f"Conversation has length {len(conversation)}, less than the needed {2 * n_turns}")
        except Exception as e:
            print(f"Error processing response: {e}")
            pass


async def main():
    semaphore = asyncio.Semaphore(500)
    tasks = []
    async with aiohttp.ClientSession() as session:
        pbar = tqdm_asyncio(total=len(base_sep_dataset["train"]))
        for i, sep in enumerate(base_sep_dataset["train"]["answer"]):
            if len(conversations) > i:
                pbar.update(1)
                continue
            if len(sep)>3000:
                sep=sep[:3000]
            n_turns = get_random_turns()
            level = get_random_level()  
            prompt = get_random_prompt(n_turns, level, sep)
            tasks.append(process_task(i, prompt, n_turns, session, semaphore, pbar))
            

        await asyncio.gather(*tasks)
        pbar.close()  # Close the progress bar

# Run the async main function in Jupyter Notebook
await main()

  0%|          | 2/11904 [00:27<38:13:36, 11.56s/it]

Conversation has length 8, less than the needed 10


  0%|          | 13/11904 [00:32<2:03:00,  1.61it/s]

Conversation has length 6, less than the needed 8


  0%|          | 14/11904 [00:32<1:52:12,  1.77it/s]

...

100%|█████████▉| 11902/11904 [2:07:32<03:40, 110.19s/it]

Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 10, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation

100%|█████████▉| 11903/11904 [2:11:18<02:24, 144.93s/it]

Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14
Conversation has length 12, less than the needed 14


100%|██████████| 11904/11904 [2:13:54<00:00,  1.48it/s] 
